# OilyGiant Mining Company Project
This project is intended to help the oil company, OilyGiant locate the best location to drill for a new well. It will collect parameters within the selected region and determine best fit based on oil quality and volume of reserves. A model will be built for predicting the volume of reserves in the new wells, determining profitability of the best wells and locating the region with the highest total profit for the potential oil wells. Only 200 will be built, among approximately 10,000 potential sites. This makes determining the region to develop particularly important, as the budget is 100m USD. 

The ultimate goal is the choose the region with the highest profit potential for the selected well locations. It will use existing data on oil samples from three regions. An analysis will be completed using Bootstrapping to determine profit and risk potential. 

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from numpy.random import RandomState

## Download data

In [2]:
#Geological exploration data for the three regions
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
#taking a first look at the data
data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


All of the dataframes appear to be in the correct datatype per column and a format that we can work with. The problem we will face upon modeling is the id column containing string values. This will confuse the model and they will need to be removed from the feature sets. 

In [9]:
#removing the id column from each DataFrame
data_0 = data_0.drop(columns=['id'])
data_1 = data_1.drop(columns=['id'])
data_2 = data_2.drop(columns=['id'])

## Model Training

We will use a LinearRegression model in this study. The primary reason for this is the target variable, being a continuous outcome variable in thousands of barrels of oil reserves. This model is also preferred over other models like decision trees, as the relationship between the target variables is anticipated to be linear and not complex in nature. 

In [10]:
#creating a function to handle the RMSE and Predicted Average Reserves
def model_region(data):
    
    #preparing features and target
    features = data.drop(['product'], axis=1)
    target = data['product']
    
    #splitting the data
    X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.25, random_state=42)
    
    #train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    #make predictions
    predictions = model.predict(X_val)
    mse = mean_squared_error(y_val, predictions)
    rmse = mse ** 0.5
    
    return rmse, predictions.mean()

datasets = [data_0, data_1, data_2]

for i, data in enumerate(datasets):
    rmse, avg_pred = model_region(data)
    print(f"Region {i}: RMSE = {rmse:.2f}, Average Predicted Reserves {i} = {avg_pred:.2f}")

Region 0: RMSE = 37.76, Average Predicted Reserves 0 = 92.40
Region 1: RMSE = 0.89, Average Predicted Reserves 1 = 68.71
Region 2: RMSE = 40.15, Average Predicted Reserves 2 = 94.77


### Conclusions from Regional Analysis
We have predicted that the mean reserves to be found within region 0 are 92.70 thousand barrels per unique well. This is a good result compared to region 1, with much higher average reserves. The error of 37.83 is something to consider, as this will help to identify the risk of allocating resources to this region. 

The average reserves in region 1 to be much lower than the other two regions analyzed. This comes with the much lower RMSE score of 0.89 however, and should be considered carefully in the final recommendation. This lower error would allow financial metrics and profitability forecasting to be much more precise than the other two regions within this study.

Region 2 has been predicted to have the highest average oil reserves per well. This high average also comes with the risk of a higher error risk, as the RMSE is 39.73. This means that the risk of drilling in this region is higher, and the potential for highest profitability is still questionable due to the risk involved with less than favorable results from drilling a well with lower than expected oil volume. 

In [11]:
#saving the predictions for each individual region
features0 = data_0.drop(['product'],axis=1)
target0 = data_0['product']

model = LinearRegression()

#region 0:

features0_train, features0_valid, target0_train, target0_valid = train_test_split(features0,
        target0, test_size=0.25, random_state=123)

model.fit(features0_train, target0_train)
predictions_0 = model.predict(features0_valid)

#reset index for future bootstrapping
target0_valid = target0_valid.reset_index(drop=True)

In [12]:
#region 1:

features1 = data_1.drop(['product'],axis=1)
target1 = data_1['product']

features1_train, features1_valid, target1_train, target1_valid = train_test_split(features1, 
        target1, test_size=0.25, random_state=123)

model.fit(features1_train, target1_train)
predictions_1 = model.predict(features1_valid)

#reset index for future bootstrapping
target1_valid = target1_valid.reset_index(drop=True)

In [13]:
#region 2:

features2 = data_2.drop(['product'],axis=1)
target2 = data_2['product']

features2_train, features2_valid, target2_train, target2_valid = train_test_split(features2, 
        target2, test_size=0.25, random_state=123)

model.fit(features2_train, target2_train)
predictions_2 = model.predict(features2_valid)

#reset index for future bootstrapping
target2_valid = target2_valid.reset_index(drop=True)

In [14]:
#checking the shape of each set
print(f"Region 0 shape:")
print(features0_train.shape)
print(features0_valid.shape)

print(f"Region 1 shape:")
print(features1_train.shape)
print(features1_valid.shape)

print(f"Region 2 shape:")
print(features2_train.shape)
print(features2_valid.shape)

Region 0 shape:
(75000, 3)
(25000, 3)
Region 1 shape:
(75000, 3)
(25000, 3)
Region 2 shape:
(75000, 3)
(25000, 3)


## Profit Calculations

#### Summary of profit requirements

Here are the factors to consider for turning a profit with the oil well development project. 
> The total budget is 100m USD.<br>
The number of wells required is 200.<br>
Each well must achieve a profit, on average, of 100m/200 = 500,000 USD<br>
At 4.5 USD per barrel, that is 500,000/4.5 = 111,111.11 barrels, or 111.11 units of product per location.

#### Profit calculation considerations prior to analysis

We can see that region 0 and 2 have the highest average reserves, but finding/analyzing the top performing well locations will give us a better idea of how profitable the overall region will be. Because we only need 200 wells, it can be assumed that avoiding the lower volume locations can be achieved with enough regularity to achieve profitability. 

This does come with risk, however, as the difference between the average oil reserve for each region is considerably lower than the required output of 111 thousand barrels per well. Further analysis is required to make the best recommendation.

#### Calculating Profit with a Function

In [15]:
#saving the glabal variables prior to the function builds

#total project budget
BUDGET = 100000000

#number of wells to sample in bootstrapping
n_boot = 500

#revenue per product unit
REVENUE_PER_THOUSAND_BARRELS = 4500

#number of wells to be developed
n_wells = 200

#budget per well
BUDGET_PER_WELL = BUDGET / n_wells

#top 200 wells
#def top_200(target, predictions):
    #top_wells = predictions.sort_values(ascending=False).head(200).index
    

In [16]:
# pick wells with the highest volume of predictions, then sort prediction values in descending order:
def func_profit(target, predictions, count):
    # pick the best predicted wells and return their indices:
    top_wells= np.argsort(predictions)[::-1]
    top_indices = top_wells[:count]
    # find the corresponding reserves.
    total_reserves = target.iloc[top_indices].sum()
    
    #calculate the revenue
    revenue = total_reserves * REVENUE_PER_THOUSAND_BARRELS
    profit = revenue - BUDGET
    return profit

#### Estimated profit per region

In [17]:
print('Profit from region 0:', func_profit(target0_valid, predictions_0, 200))
print('Profit from region 1:', func_profit(target1_valid, predictions_1, 200))
print('Profit from region 2:', func_profit(target2_valid, predictions_2, 200))

Profit from region 0: 35346709.17261383
Profit from region 1: 24150866.966815114
Profit from region 2: 23703438.630213737


Based on the findings from the profit calculation of the top 200 sites within each region, we can assume that region 0 will likely result in the most profitable development of wells. Even with a higer risk of lower producing wells, the margin from 32 million to the second best at 28 million leaves a comfortable space for the decision.

### Results from well comparison study

Even though region 1 has lower average reserves, it beat out region 2 for estimated profits. This is due to the slightly higher frequency of top performing reserves within the set. This is particularly interesting because of the much lower RMSE of the region 1 analysis. I can say with confidence at this point, only regions 0 and 1 are being considered for recommendation. 

### Using Bootstrapping to find the distribution of profit

In [18]:
#create a random number generator using RandomState
state = RandomState(12345)

def bootstrapping(target, predictions, count, rev_per_unit, budget):
    values = []
    probs = pd.Series(predictions, index=target.index)
    for i in range(1000):
        # Sample
        target_subsample = target.sample(n=n_boot, replace=True, random_state=state)
        probs_subsample = probs[target_subsample.index]
        
        # Getting top indexes after predictions are sorted
        top_wells = probs_subsample.sort_values(ascending=False).head(count).index
        total_reserves = target_subsample.loc[top_wells].sum()
        
        # Calculating profits
        revenue = total_reserves * REVENUE_PER_THOUSAND_BARRELS
        profit = revenue - BUDGET
        
        values.append(profit)
        
    values_series = pd.Series(values)
    risk_of_losses = (values_series < 0).mean() * 100
    upper = values_series.quantile(0.975)
    lower = values_series.quantile(0.025)
    mean = values_series.mean()
    
    
    print('The lower and upper bounds are', [lower, upper])
    print('The average profit is', mean)
    print('The risk of losses is', round(risk_of_losses, 2), '% - This is the probability that the profit will be negative')
    print()
    return values_series

In [19]:
#region 0 results
summary0 = bootstrapping(target0_valid, predictions_0, n_wells, REVENUE_PER_THOUSAND_BARRELS, BUDGET)


The lower and upper bounds are [1028835.668918955, 13746430.880370922]
The average profit is 6880752.315149353
The risk of losses is 1.2 % - This is the probability that the profit will be negative



In [20]:
#region 1 results
summary1 = bootstrapping(target1_valid, predictions_1, n_wells, REVENUE_PER_THOUSAND_BARRELS, BUDGET)

The lower and upper bounds are [2272168.9561553868, 11948939.098337654]
The average profit is 6829668.28871909
The risk of losses is 0.1 % - This is the probability that the profit will be negative



In [21]:
#region 2 results
summary2 = bootstrapping(target2_valid, predictions_2, n_wells, REVENUE_PER_THOUSAND_BARRELS, BUDGET)

The lower and upper bounds are [-703478.5804266758, 11797268.002004158]
The average profit is 5544290.3751238985
The risk of losses is 3.7 % - This is the probability that the profit will be negative



### Conclusions from regional profit analysis by bootstrapping

The bootstrapping technique demonstrated a much closer profitability between region 0 and region 1. The very close average profit leads us to the next metric to compare, risk of loss. The much lower risk of loss in region 1 would lead me to recommend it over the slightly higher profitability potential of region 0. 

I can therefore recommend development of region 1 with a high level of confidence in the forecasting of profitability due to the very consistent oil reserves among the top 200 locations. 